In [1]:

from datasets import load_dataset


ds = load_dataset("CharlyR/vtikz", "tikz", split="test")
ds = ds.select_columns(["id","type","instruction","code","image_solution","image_input","code_solution"])


/home/creux/Documents/AI/VIFagent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from openai import OpenAI
import os
from google import genai
from google.genai import types as genTypes
from vif.falcon.edition import OracleEditionModule
from vif.falcon.falcon import Falcon
from vif.falcon.oracle.guided_oracle.guided_code_oracle import OracleGuidedCodeModule
from vif.utils.renderer.tex_renderer import TexRenderer

gclient = genai.Client(
    api_key=os.environ.get("GOOGLE_API_KEY"),
    http_options=genTypes.HttpOptions(api_version="v1alpha"),
)

oclient = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ.get("OPENROUTER_API_KEY"),
)


oracle_module = OracleGuidedCodeModule(
    model="google/gemini-2.5-flash-lite",
    temperature=0.3,
    client=oclient,
    visual_client=gclient,
    visual_generation_content_config=genTypes.GenerateContentConfig(temperature=0.3),
    visual_model="gemini-2.5-pro"
)
edition_module = OracleEditionModule(
    model="google/gemini-2.5-flash-lite",
    temperature=0.3,
    client=oclient,
)

agent = Falcon(
    code_renderer=TexRenderer().from_string_to_image,
    oracle_module=oracle_module,
    edition_module=edition_module,
    observe=True,
    observe_folder="/home/creux/Documents/AI/VIFagent/.tmp/observe",
)

You are trying to use a model that was created with Sentence Transformers version 5.0.0, but you're currently using version 4.1.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.
2025-08-12 15:51:18.210 | WARNING  | vif.falcon.falcon:__init__:46 - The observe parameter is activated, dict will be stored at /home/creux/Documents/AI/VIFagent/.tmp/observe/1208-15:51:18.pickle


In [3]:
agent.apply_instruction(ds[0]["code"],ds[0]["instruction"],ds[0]["id"])

2025-08-12 15:51:19.614 | INFO     | vif.falcon.falcon:apply_instruction:65 - Creating the oracle
2025-08-12 15:51:19.616 | INFO     | vif.falcon.oracle.guided_oracle.guided_code_oracle:get_oracle:101 - Creating Oracle for instruction Switch the color of the circle and the triangle
2025-08-12 15:51:23.041 | INFO     | vif.falcon.oracle.guided_oracle.guided_code_oracle:get_oracle:116 - Oracle created: def test_valid_customization() -> bool:
    # The original image contains a red triangle, a green circle, & a blue square.
    # The prompt asks to switch the colors of the circle & the triangle.
    # This means the triangle should become green & the circle should become red.
    return color("triangle", "green") & color("circle", "red")

2025-08-12 15:51:23.354 | INFO     | vif.falcon.edition:customize:115 - Sending initial message
2025-08-12 15:51:26.515 | INFO     | vif.falcon.edition:customize:124 - LLM response:ChatCompletion(id='gen-1755006683-50vPgZZJSjrNtQZfykum', choices=[Choice(

KeyboardInterrupt: 

In [ ]:
print(ds[0]["instruction"])
ds[0]["image_input"]